In [ ]:
import timm
import torch
import wandb
import fastai
from fastai.callback.wandb import WandbCallback
from fastai.vision.all import *

In [ ]:
config = SimpleNamespace(
    batch_size=2,
    img_size=224,
    seed=42,
    pretrained=True,
    model_name="maxvit_rmlp_small_rw_224.sw_in1k", # try with maxvit_nano_rw_256.sw_in1k # regnetx_040 coatnet_bn_0_rw_224.sw_in1k
    epochs=3)

def get_planets(batch_size=8, img_size=224, seed=42):
    "A sample of the planets dataset"
    dataset_path=untar_data(URLs.PLANET_SAMPLE)
    dls = ImageDataLoaders.from_csv(dataset_path, 
                                    folder="train", 
                                    csv_fname="labels.csv",
                                    label_delim=" ",
                                    suff=".jpg",
                                    bs=batch_size,
                                    seed=seed,
                                    item_tfms=Resize(img_size))
    return dls

def train(config):
    "Train the model using the supplied config"
    dls = get_planets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project="PROJECT", group='ambrosia_symbiosis', job_type='test_training', config=config):
        cbs = [MixedPrecision(), WandbCallback(log='all')]
        learn = vision_learner(dls, config.model_name, metrics=[accuracy_multi], #error_rate, 
                               cbs=cbs, pretrained=config.pretrained)
        # learn.remove_cb(ProgressCallback)
        learn.fine_tune(config.epochs)